# Домашнее задание

In [2]:
!pip install implicit

     |████████████████████████████████| 71 kB 5.4 MB/s eta 0:00:011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.5.2-cp39-cp39-macosx_10_15_x86_64.whl size=745155 sha256=fc897b951dc51e71bf0dab62e91264a891a7b445f444f59f05d3845711035cd8
  Stored in directory: /Users/ilya.ivolgin/Library/Caches/pip/wheels/71/86/ba/b0034e077749cd93a0e3489ce2565e71c706386635e11e8eb8
Successfully built implicit


In [3]:
# Загрузка библиотек

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
# Загрузка и проверка данных

data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

---

In [6]:
# Создадим df с покупками на тестовом датасете за последение 3 недели

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В data_test {} юзеров'.format(test_users))
print('В data_test {} новых юзеров'.format(new_test_users))

В data_test 2042 юзеров
В data_test 0 новых юзеров


In [7]:
# Функция random recommendation

def random_recommendation(items, n = 5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size = n, replace = False)
    
    return recs.tolist()

---

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size = n, p = weights)
    
    return recs.tolist()

In [9]:
%%time

items_weights = data_train[['item_id','sales_value']]
items_weights.loc[items_weights['sales_value'] == 0, 'sales_value'] = 0.001
items_weights['weight'] = np.log(items_weights['sales_value'])
items_weights.loc[(items_weights['weight'] < 0)] = 0

min_value = items_weights['weight'].min()
max_value = items_weights['weight'].max()
sum_ = items_weights['weight'].sum()

items_weights = items_weights[['item_id','weight']].groupby('item_id').sum()
items_weights['weight'] = items_weights['weight'] / sum_
items_weights['item_id'] = items_weights.index

/Users/ilya.ivolgin/opt/miniconda3/envs/geek/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 250 ms, sys: 79.3 ms, total: 329 ms
Wall time: 336 ms


In [10]:
# Проверка результата

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n = 5))

result.head(10)

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1045779, 1017201, 1013503, 6544236, 9859217]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[909666, 6772746, 951190, 1048507, 899624]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1016782, 890909, 6534178, 1072799, 970030]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5573964, 1081177, 13189726, 1131034, 1050425]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[7154535, 986943, 1006414, 907945, 13158498]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[883186, 6534178, 862139, 985999, 1106523]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[9835880, 1055155, 917646, 1076769, 833494]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[1038463, 13877075, 991592, 1133312, 6979427]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[1134232, 976199, 984208, 16122432, 878996]"
9,16,"[1062973, 1082185, 13007710]","[1130375, 848278, 1013192, 6533293, 962161]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [14]:
# Функция для конвертации str в int

def str_list_to_int_list(row):
    
    list_int = []
    
    for i in row.strip('][').split():
        
        list_int.append(int(i.rstrip(',')))
        
    return list_int

In [15]:
result = pd.read_csv('predictions_basic.csv')

for col in result.iloc[:,1:]:
    result[col] = result.apply(lambda row: str_list_to_int_list(row[col]), axis = 1)
    
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [16]:
# Функция precision@K

def precision_at_k(recommended_l, bought_l, k = 5):
    
    bought_l = np.array(bought_l)
    recommended_l = np.array(recommended_l)
    bought_l = bought_l
    recommended_l = recommended_l[:k]
    flags = np.isin(bought_l, recommended_l)
    precision = flags.sum() / len(recommended_l)
    
    return precision

In [17]:
# Проверка

for col in result.iloc[:,2:]:
    print(col +": ","{0:.4f}".format(result.apply(lambda row: precision_at_k(row[col], row['actual'], k = 5), axis = 1).mean()))

random_recommendation:  0.0008
popular_recommendation:  0.1552
itemitem:  0.1369
cosine:  0.1329
tfidf:  0.1390
own_purchases:  0.2019


/var/folders/wf/0ws1q6ks6yv_xwm2_24s254h0000gn/T/ipykernel_6625/2501552769.py:10: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_l)


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [9]:
# your_code